In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
import cv2
import numpy as np
import pymouse 
# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(30, 30),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Initialize video capture
cap = cv2.VideoCapture(0)

# Read the first frame
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Set initial tracking point
p0 = None

# Mouse callback function for initial point selection
def select_point(event, x, y, flags, param):
    global p0
    if event == cv2.EVENT_LBUTTONDOWN:
        p0 = np.array([[x, y]], dtype=np.float32)

# Create a window and bind the mouse callback function
cv2.namedWindow('Frame')

# Initialize PyMouse
mouse = PyMouse()

while True:
    # Read the current frame
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display instructions for selecting the initial point
    cv2.putText(frame, "Select the initial point by clicking", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # If the initial point is selected, proceed with tracking
    if p0 is not None:
        # Calculate optical flow using Lucas-Kanade algorithm
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        # Select good points
        good_new = p1[st == 1]
        good_old = p0[st == 1]

        # Draw the tracking points and motion vectors
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 255, 0), -1)
            frame = cv2.line(frame, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)

        # Update the previous frame and points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Check for mouse click event
    if p0 is None:
        mouse_event = mouse.get_mouse_event()
        if mouse_event and mouse_event['event_type'] == 'down':
            x, y = mouse_event['position']
            p0 = np.array([[x, y]], dtype=np.float32)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'windows'